In [2]:
%cd ~/cdv/

import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

import numpy as np
import pandas as pd

/home/nmiklaucic/cdv


/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import jax 
import jax.numpy as jnp
import jax.random as jr

In [ ]:
https://github.com/MDIL-SNU/SevenNet/raw/afb56e10b6a27190f7c3ce25cbf666cf9175608e/sevenn/pretrained_potentials/SevenNet_0__11July2024/checkpoint_sevennet_0.pth

In [7]:
data = np.load('data/descriptors.npy', allow_pickle=True)
data

array([{'Bi': array([-0.5519185 ,  0.38967603,  0.143172  ,  0.01463768, -0.24741472,
               0.7029988 ,  0.43916857,  0.78207815,  0.19009823, -0.6162896 ,
              -0.8624371 , -0.05240107,  0.2627563 , -0.46142912,  0.4094873 ,
               0.45051962,  0.09522059,  0.3535585 ,  0.03430389, -0.1802409 ,
               0.53339267,  0.40477946,  0.45597133, -0.4825731 ,  0.09303026,
               0.07149379,  0.5064508 ,  0.08615639,  0.18724123,  0.37015647,
               0.2677351 ,  0.04265396, -0.22176658,  0.06443211,  0.22288387,
               0.08123558,  0.15853623,  0.08601445, -0.43910974,  0.42409682,
              -0.2185232 , -0.4421243 ,  0.25676396,  0.07853271, -0.07904776,
               0.48440218, -0.1998131 ,  0.12854037,  0.19379508, -0.21508032,
               0.00817126,  0.19234861, -0.07130957,  0.5616611 , -0.2438547 ,
              -0.5371894 ,  0.33045822,  0.20939445, -0.02393016,  0.34265345,
              -0.20162535,  0.11994956, -0.23

In [20]:
data[2]['Li']

array([-0.11615909, -0.19692884,  0.19429594, -0.05228885,  0.03560692,
       -0.23269777, -0.0206372 , -0.16500407,  0.0880481 ,  0.13402446,
        0.00803491, -0.00744784, -0.27847075,  0.112648  , -0.0114516 ,
        0.02901453, -0.01915408,  0.01409212, -0.04032236,  0.01144508,
        0.0537791 ,  0.23947455,  0.08187871,  0.04036358, -0.20416553,
       -0.19185548, -0.08966797,  0.00860815, -0.19952011,  0.00394893,
       -0.20265307,  0.10348865,  0.2223482 ,  0.21389896, -0.01703147,
       -0.24417868, -0.17206405,  0.12814513,  0.04141722,  0.29710904,
       -0.15083289, -0.35181856,  0.07602337,  0.11751472, -0.13372968,
       -0.0444153 , -0.45691687,  0.06347239,  0.23687334, -0.10741295,
        0.1281746 ,  0.30525416, -0.3071004 , -0.10051287, -0.08454067,
       -0.3027175 ,  0.1135361 ,  0.05216132,  0.01216342, -0.08096419,
       -0.12845607, -0.1592029 ,  0.07083105, -0.20810927, -0.08720636,
       -0.20302151,  0.17167005, -0.36179146,  0.11968561,  0.16

In [21]:
data[6]['Li']

array([-6.00118190e-02,  1.82503350e-02,  1.82828993e-01,  1.38917461e-01,
        2.35016897e-01, -1.88426554e-01, -1.05710596e-01, -8.36274326e-02,
       -1.36471868e-01,  1.38384849e-01,  5.54084145e-02, -9.60081443e-02,
       -2.94414043e-01,  2.62277514e-01,  1.27042755e-01,  2.17888623e-01,
       -8.23992714e-02,  1.37415469e-01, -9.53201801e-02, -5.29757515e-02,
        9.68071744e-02,  2.25796446e-01,  2.88386703e-01,  2.23183334e-01,
       -6.04887754e-02, -1.43441372e-03, -6.74297940e-03,  2.98914518e-02,
        8.89455341e-03,  2.36792099e-02,  5.23536392e-02,  2.16914713e-01,
       -1.45019796e-02,  2.05293685e-01, -1.55319840e-01, -3.56260657e-01,
       -4.82844859e-02, -3.76168825e-02,  4.45358679e-02,  1.17648005e-01,
        5.36587983e-02, -3.55329603e-01,  8.71642679e-02, -1.20517341e-02,
       -9.38353166e-02, -2.58599631e-02, -3.40167522e-01, -2.62726665e-01,
        4.27742511e-01, -1.97894454e-01, -1.09595597e-01,  1.03725217e-01,
       -2.93157995e-01,  

In [3]:
from jax.experimental import mesh_utils
from jax.sharding import Mesh, PartitionSpec as P, NamedSharding

d = 3
mesh = Mesh(mesh_utils.create_device_mesh((d,), devices=jax.devices()[:d]), 'batch')
sharding = NamedSharding(mesh, P('batch', None))

In [4]:
tree = {
    'a': jr.normal(jr.key(123), (3, 64)),
    'b': jr.normal(jr.key(123), (3, 32, 32)),
    'c': jr.normal(jr.key(123), (3, 1))
}

tree = jax.device_put(tree, sharding)

jax.debug.visualize_array_sharding(tree['a'][0])

E0816 16:33:08.249164 1082658 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.
E0816 16:33:08.249187 1082656 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.
E0816 16:33:08.249202 1082654 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:3

XlaRuntimeError: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.: while running replica 0 and partition 0 of a replicated computation (other replicas may have failed as well).

: 